<img src="figuras/cabecera.png" alt="Drawing" style="width: 1100px;"/>

# EJERCICIO 3
# Aprendizaje no supervisado: Clustering.

## *Clustering de consumidores*


En el aprendizaje no supervisado, la tarea clásica es el **análisis de clusters** (grupos) en el que se encuentran patrones o grupos ocultos en los datos. La mayoría de las veces las tareas de aprendizaje no supervisado tienen una *solución abierta*, por lo que hay que interpretar los resultados y comprobar si tienen sentido.

**Objetivo:** En este ejemplo se utilizan datos que contienen información acerca del consumo eléctrico de un grupo de consumidores eléctricos. El objetivo es encontrar el número óptimo de clusters para agrupar los diferentes patrones de consumo diarios. El resultado se utilizará para fines comerciales y estratégicos.

### Antes de empezar:

* En el archivo **clustering-consumos.xlsx** se encuentra el conjunto de datos de entrada de este ejemplo (atributos). 
* **NO** existen las etiquetas en el Aprendizaje **NO Supervisado**. 


<img src="figuras/no-supervisado.png" alt="Drawing" style="width: 600px;"/>


## **1. Importar librerías y datos**

In [ ]:
# Importar librerías
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Librería de visualización de datos
plt.style.use('seaborn')

# Seleccionamos las columnas que necesitamos
df_consumos = pd.read_excel('dataset/clustering-consumos.xlsx')
df_consumos

## **2. Comprender los datos**

Es necesario visualizar y comprender los datos con los que vamos a trabajar, así como conocer sus características. 

1. ¿Cuántos datos hay? ¿Cuántos atributos hay en los datos?  
2. ¿Qué significan?
3. ¿Falta algún dato?
4. Resumen estadístico del conjunto de datos de entrada.

In [ ]:
# Dimensión de los datos de entrada (filas x columnas)
df_consumos.shape

In [ ]:
# Veamos como es la apariencia de los datos
df_consumos.head()

In [ ]:
df_consumos.tail()

In [ ]:
# Pongo como índice el número de CUP 
df_consumos.set_index('CUPs', inplace = True)
df_consumos

In [ ]:
# Se comprueba si existe algún dato categórico que haya que transformar
df_consumos.dtypes

**3. ¿Falta algún dato?** Se comprueba si falta algún dato, y de ser así, se realiza el recuento de celdas vacías en cada atributo. En este caso, no falta ningún dato en el conjunto de datos de entrada (no existen valores *Nan*).

In [ ]:
df_consumos.isna().sum()

In [8]:
# Imputación de datos con pandas
df_consumos.fillna(method='ffill', inplace=True)

# Imputación de datos con Scikitlearn (media, mediana, valor más frecuente...)
# from sklearn.impute import SimpleImputer
# Mas info: https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer

In [ ]:
# Comprobamos que se han imputado los valores correctamente
df_consumos.isna().sum()

In [ ]:
df_consumos

**4. Resumen estadístico del conjunto de datos de entrada:** La estadística descriptiva recolecta y analiza el conjunto de datos de entrada con el objetivo de describir las características y comportamientos de este conjunto mediante las siguientes medidas resumen: número total de observaciones (count), media (mean), desviación estándar (std), valor mínimo (min), valor máximo (max) y los valores de los diferentes cuartiles (25%, 50%, 75%).

In [ ]:
# Evaluamos la naturaleza de los datos con datos estadísticos descriptivos
df_consumos.describe()

## **3. Visualizar los datos**

Una manera visual de entender los datos de entrada. 
1. Histograma
2. Curva de densidad
3. Boxplots


**1. Histograma**

Respresentación gráfica de cada uno de los atributos en forma de barras, donde la superficie de la barra es proporcional a la frecuencia de los valores representados.

In [ ]:
histograma = df_consumos.hist(xlabelsize=10, ylabelsize=10, bins=100, figsize=(18, 12))

**2. Gráfico de densidades**

Visualiza la distribución de los datos. Es una variable del histograma, pero elimina el ruido, por lo que son mejores para determinar la forma de distribución de un atributo. Lo spicos del gráfico de densidad ayudan a mostrar dónde los valores se concentran más. 

In [ ]:
density = df_consumos.plot(kind='kde', legend=True, layout=(1, 1), figsize=(18, 12),
                        fontsize=20, stacked=True) 

**3. Boxplots** 

El boxplot (diagrama de caja) nos permite identificar los valores atípicos y comparar distribuciones. Además, se conoce como se distribuyen el 50% de los valores (dentro de la caja). 

In [ ]:
boxplot = df_consumos.plot(kind='box', legend=True, layout=(1, 1), figsize=(18, 12),
                        fontsize=16, stacked=True) 


## *4. Preparar los datos*

1. Escalar datos. 


### Ploteamos los datos
El gráfico muestra el consumo horario de un grupo de consumidores durante un día.

In [ ]:
# Consumo horario
df_consumos.T.plot(figsize=(18, 8), title='Consumo diario', legend=False, color='blue', alpha=0.05, 
                   fontsize=15, xlim=[0,23], ylabel='Energía Consumida [kWh]')

**1. Escalar los datos**. 

Se escalan los datos utilizando el método de *MinMaxScaler()*

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = df_consumos.values.copy()
X_scale = pd.DataFrame(scaler.fit_transform(X))
X_scale.head()

## 5. Construcción del modelo de aprendizaje NO supervisado: Clustering de consumos utilizando K-means

Se agrupan los datos utilizando el algoritmo [K-Means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

El algoritmo K-means necesita que se le indique el número de clústers en que se quieren agrupar los datos. Se ejecuta el algoritmo para varios clusters y luego se comparan los resultados utilizando el método Elbow, que indicará el número óptimo de clusters.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

elbow_method = []

# Evalúo el algoritmo K-means para un rango de [2,10] clústers 
n_cluster_list = range(1,15)
print(list(n_cluster_list))

### ¿Cómo saber el número óptimo de clusters? Con el método de Elbow.
Se utiliza el [Método de Elbow] para ayudarnos a elegir el número óptimo de clusters. 

* Este método utiliza los valores de la inercia obtenidos tras aplicar el K-means a diferente número de Clusters (desde 1 a N Clusters), siendo la inercia la suma de las distancias al cuadrado de cada objeto del Cluster a su centroide.
* Para hacer uso de este método partimos del cálculo de la distorsión promedio de cada clúster, esto es la distancia de cada elemento con su centroide correspondiente.
* Buscamos la parte de la gráfica donde la línea es menos suave o cambia abruptamente lo que forma un “codo”.

[Método de Elbow]: https://jarroba.com/seleccion-del-numero-optimo-clusters/


<img src="figuras/elbow-method.png" alt="Drawing" style="width: 800px;"/>

In [ ]:
import matplotlib.pyplot as plt

# Iteración para evaluar K-means para diferentes números de clusters (n_clusters)
for n_cluster in n_cluster_list:
    kmeans = KMeans(n_clusters=n_cluster, random_state=0)
    cluster_found = kmeans.fit_predict(X_scale)
    elbow_method.append(kmeans.inertia_) 


# Gráfica del método de Elbow
plt.plot(n_cluster_list, elbow_method, 'bx-')
plt.xlabel('Número de centroides')
plt.ylabel('Within-Cluster Sum of Square')
plt.title('Método Elbow para número óptimo de clusters')
plt.show()

# El número óptimo de clusters es de k=2

In [ ]:
# Entreno el K-means para k=X, visto el resultado del método Elbow
kmeans = KMeans(n_clusters=XXXXX)
cluster_found = kmeans.fit_predict(X_scale)
cluster_found_sr = pd.Series(cluster_found, name='cluster')

# Creo un multindex del tipo: (fecha,cluster al que pertenece el día)
df_consumos = df_consumos.set_index(cluster_found_sr, append=True)

#Guardamos los clusters en un excel
df_consumos.to_excel('dataset/resultados-clusters.xlsx')

df_consumos.index

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14,5))
color_list = ['blue', 'green', 'red']
cluster_values = sorted(df_consumos.index.get_level_values('cluster').unique())

for cluster, color in zip(cluster_values, color_list):
    # ploteo todas las lineas de cada cluster
    df_consumos.xs(cluster, level=1).T.plot(ax=ax, legend=False, alpha=0.05, color=color)
    # ploteo la línea con el valor de la mediana de cada cluster
    df_consumos.xs(cluster, level=1).median().plot(ax=ax, color=color, legend=False, alpha=1, ls='--')

ax.set_ylabel('Potencia media horaria [kW]')
ax.set_xlabel('Horas')


K-means ha encontrado los los clúster con las siguientes características, mirando la gráfica anterior:
* El clúster de azul concentra los patrones de consumo menos elevados.
* El clúster rojo concentra una mayor potencia media horaria de consumo.
